[View in Colaboratory](https://colab.research.google.com/github/rutgerhofste/bq-geoms/blob/master/Y2018M09D11_RH_GADM_V01.ipynb)


Python 3 runtime

##create GADM derived table in bigquery. 
Use case: Spatial Join in Bigquery

Question on stack exchange

Notebook is Google Colab compatible.

This script will process the global geopackage. Please note that geopandas handles data in memory so make sure your VM is sufficiently large to handle the level.

In [0]:
# Select level 0 (country) - 5 (commune). See https://gadm.org/index.html for 
# more information.
LEVEL = 0

In [2]:
!pip install shapely geopandas descartes sqlalchemy pydrive psycopg2

    100% |████████████████████████████████| 1.5MB 10.9MB/s 
    100% |████████████████████████████████| 901kB 19.8MB/s 
    100% |████████████████████████████████| 5.6MB 838kB/s 
    100% |████████████████████████████████| 993kB 17.7MB/s 
    100% |████████████████████████████████| 2.7MB 984kB/s 
    100% |████████████████████████████████| 4.4MB 5.4MB/s 
    100% |████████████████████████████████| 15.8MB 2.0MB/s 
    100% |████████████████████████████████| 71kB 21.4MB/s 
  Running setup.py bdist_wheel for sqlalchemy ... - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/2b/e7/bc/03c832ebc5c8f34a4ddab35f1bc812feb1e606d1147d6af2c9
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
  Running setup.py bdist_wheel for pyproj ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \

In [3]:
# Version 3.6 Date accessed 2018 09 11
# Compressed Size = 1.2 GB 
# Uncompressed Size =  3.5 GB

!rm -r input_data
!mkdir -p input_data

rm: cannot remove 'input_data': No such file or directory


In [0]:
url = "https://biogeo.ucdavis.edu/data/gadm3.6/gadm36_levels_gpkg.zip"

In [5]:
!wget {url} -P input_data/


Redirecting output to ‘wget-log’.


In [0]:
import os
import geopandas as gpd

from google.cloud import bigquery
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [7]:
os.listdir("input_data")

['gadm36_levels_gpkg.zip']

In [8]:
!unzip 'input_data/gadm36_levels_gpkg.zip' -d input_data/

Archive:  input_data/gadm36_levels_gpkg.zip
  inflating: input_data/gadm36_levels.gpkg  
  inflating: input_data/license.txt  


In [0]:
layer = "level{:01.0f}".format(LEVEL)

In [0]:
gdf = gpd.read_file("input_data/gadm36_levels.gpkg",layer=layer)

In [11]:
gdf.head()

,GID_0,NAME_0,geometry
0,ABW,Aruba,(POLYGON ((-69.97819519042969 12.4698610305787...
1,AFG,Afghanistan,"(POLYGON ((68.52643585 31.75434685, 68.5385208..."
2,AGO,Angola,"(POLYGON ((11.73347187 -16.67255211, 11.733471..."
3,AIA,Anguilla,(POLYGON ((-63.42375183105457 18.5890274047852...
4,ALA,Åland,"(POLYGON ((21.3219471 59.74985886, 21.3219471 ..."


In [0]:
gdf["geometry_wkt"] = gdf["geometry"].apply(lambda x: x.to_wkt())

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Google Drive ID of the google bigquery creds.
# change to your own credentials.

BQ_CREDS_ID = "1JzjuosHbtV7mzE0f85ZTXsNc2X4p8BFK"
BQ_PROJECT_ID = "aqueduct30"

OUTPUT_DATASET = "spatial_test" #create dataset first if not exist.
OUTPUT_TABLE_NAME = "gadm_{}_wkt".format(layer)

bq_file = drive.CreateFile({'id': BQ_CREDS_ID}).GetContentString()
client = bigquery.Client(project=BQ_PROJECT_ID)

In [0]:
destination_table = "{}.{}".format(OUTPUT_DATASET,OUTPUT_TABLE_NAME)

In [0]:
df = gdf.drop("geometry",axis=1)

'MULTIPOLYGON (((-69.9781951904296875 12.4698610305787270, -69.9784698486328125 12.4698610305787270, -69.9784698486328125 12.4701385498046307, -69.9790267944335938 12.4701385498046307, -69.9790267944335938 12.4706945419310955, -69.9787521362304688 12.4706945419310955, -69.9787521362304688 12.4709739685059731, -69.9784698486328125 12.4709711074829102, -69.9784698486328125 12.4712486267089844, -69.9779129028320312 12.4712486267089844, -69.9779129028320312 12.4715299606324379, -69.9776382446289062 12.4715299606324379, -69.9776382446289062 12.4720830917358398, -69.9779129028320312 12.4720830917358398, -69.9779129028320312 12.4723615646362305, -69.9784698486328125 12.4723615646362305, -69.9784698486328125 12.4726400375366211, -69.9790267944335938 12.4726400375366211, -69.9790267944335938 12.4729175567626953, -69.9793090820311363 12.4729175567626953, -69.9793090820311363 12.4731960296630859, -69.9801406860350994 12.4731931686401936, -69.9801406860350994 12.4734706878663246, -69.9809722900390

In [0]:
#df.to_csv("df.csv")
#f = drive.CreateFile()
#f.SetContentFile('df.csv')
#f.Upload()

In [36]:
df.shape

(40, 3)

In [35]:
df.to_gbq(destination_table=destination_table,
          project_id = BQ_PROJECT_ID,
          if_exists="replace",
          chunksize=2)

GenericGBQException: ignored

Now the data is in WKT on bigquery. You can visualize using:
https://bigquerygeoviz.appspot.com/

the last step would be to convert the WKT to geometry on bigquery. 
SELECT *, ST_GeogFromText(geometry_wkt) AS g FROM `aqueduct30.spatial_test.gadm_levelx_wkt`

please note that that the hexagons will be in Geopraphy instead of geometry!

happy mapping.

